# Computation in TensorFlow

    John-Mark Agosta
    9 March 2018
    
This basic tutorial demonstrates running basic matrix operations in TensorFlow for solving a simple decision problem. TensorFlow has the ability to do matrix math, but it first creates a computation graph (the "flow"), then evaluates it. Understanding how TensorFlow runs computations is basic to understanding its uses in learning and optimization. 

We show first how values can be inspected 

In [108]:
import tensorflow as tf

In [109]:
# Bayes rule to find P(s | x) from the observed sample of x

# The prior
PRIOR_ON_S = 0.3
p_s = tf.constant([PRIOR_ON_S, 1-PRIOR_ON_S],name='S', dtype= tf.float32)
# The likelihood, P ( X | S )
x_discrete = tf.constant([[0.9, 0.1], [0.4, 0.6]], name= 'X_S', dtype= tf.float32)
# Multiply the two conformally, to get the joint
# Note there is no automatic type coercion. 
xs_joint = tf.multiply(tf.transpose(x_discrete),tf.cast(tf.reshape(tf.tile(p_s, [2]), [2,2]), tf.float32))
# Sum along the observable as the normalizer.
s_marginal = tf.reduce_sum(xs_joint, 1)
# Division to get the posterior. Note how the marginal is "broadcast, so no tiling is needed."
s_x = tf.realdiv(tf.transpose(xs_joint), s_marginal)
# Check that the posterior sums to one. 
s_test = tf.reduce_sum(s_x, 0)
print("prior :", p_s.eval())
print("likelihood :", x_discrete.eval())
print("joint on x,s :", xs_joint.eval())  
print("s marg :", s_marginal.eval())
print("s posterior :", s_x.eval())
print("Check normalization :",s_test.eval())
session.close()

prior : [ 0.30000001  0.69999999]
likelihood : [[ 0.89999998  0.1       ]
 [ 0.40000001  0.60000002]]
joint on x,s : [[ 0.27000001  0.28      ]
 [ 0.03        0.42000002]]
s marg : [ 0.55000001  0.45000002]
s posterior : [[ 0.4909091   0.06666667]
 [ 0.5090909   0.93333334]]
Check normalization : [ 1.  1.]


In [110]:
# Using the session graph instead. 
# Note we don't need to re-specify the terms created for the interactive session. 
#get the tensorflow session
with tf.Session() as the_session:
    # Initialize all variables - a convenience not to have call the initializer on each individually.
    sess.run(tf.initialize_all_variables())
    # Run the graph and "fetch" the final values. 
    posterior, marginal = the_session.run([s_x, s_marginal])
print("Posterior P(S | X):", posterior)
print("Marginal P(S):", marginal)

Posterior : [[ 0.4909091   0.06666667]
 [ 0.5090909   0.93333334]]
Marginal : [ 0.55000001  0.45000002]


The full decision problem takes four variables, a value, _V_, state, _S_, observation, _X_, and decision, _D_.  The _V_ function is a table indexed by the two binary variables, _S_, a Bernoulli variable, and _D_. _X_ is normal, conditioned on _S_, distributed _P( X | S )_. The computation solves for _D_ as a function of _X_ that maximizes the expected value of _V_.

In [113]:
# The value matrix V(d, s) is multiplied by P(S | X) to give a 3-dim tensor, in d, s, x
v = tf.constant([[3,2], [9, 0]], name='V', dtype= tf.int32)
# Expand it to d, s, x
v = tf.tile(v, v)
print(v.eval())
d = tf.Variable([[0,0]], dtype=tf.float32, name='D')

ValueError: Shape must be rank 1 but is rank 2 for 'Tile_37' (op: 'Tile') with input shapes: [2,2], [2,2].

In [114]:
tf.tile

SyntaxError: invalid syntax (<ipython-input-114-2b0798c2d6f2>, line 1)

In [111]:
# Here's another version, with Gaussian noise instead. 
# Define the variables
REPETITIONS = 5
TRUE_S = 0.1
#tf.global_variables_initializer()
bernoulli = tf.contrib.distributions.Bernoulli(probs= [TRUE_S])
s = bernoulli.sample(REPETITIONS)
noise = tf.distributions.Normal(0.0,1.0)
err = noise.sample(REPETITIONS)
# Without the transpose, add performs an outer-product instead of term-by-term addition
s_plus_noise = tf.add(tf.cast(tf.transpose(s), tf.float32), err)

In [112]:
# InteractiveSession is a convenience to evaluate TensorFlow objects interactively. 
# This is largely a debugging aid. 
session = tf.InteractiveSession()
print("v :",v.eval())
print("s :",s.eval())
print("s + x", s_plus_noise.eval())

v : [[3 2]
 [9 0]]
s : [[0]
 [0]
 [1]
 [0]
 [0]]
s + x [[-1.16575384 -0.50427359 -0.71540195 -0.80070239 -1.5347358 ]]
